# Optimointi pythonilla SciPy:n linprog-funktiolla
Optimointia voi myös tehdä tietokoneen avulla. Tässä tapauksessa käytetään pythonia. Tätä varten tarvitset **SciPy** libraryn ja sieltä seuraavat funktiot.

In [ ]:
import numpy as np
from scipy.optimize import linprog

# Funktio
Käytämme samaa esimerkkiä kuin aikaisemmin, eli rakennamme tuoleja ja pöytiä legoilla.

In [ ]:
print("\nHei! Olen Optimointikone, ja ratkaisen optimointiongelman sinun puolestasi! Tarvitsen vain vastauksen muutamaan kysymykseen, jotta voin maksimoida kohdefunktion...")

lego_pieni_määrä = int(input("Kuinka paljon sinulla on pieniä legoja käytössä? "))
lego_iso_määrä = int(input("Kuinka paljon sinulla on isoja legoja käytössä? "))

lego_pieni_hinta = int(input("Mikä on pienen legon hinta? "))
lego_iso_hinta = int(input("Mikä on ison legon hinta? "))

tuoli_myyntihinta = int(input("Mikä on tuolin myyntihinta? "))
pöytä_myyntihinta = int(input("Mikä on pöydän myyntihinta? "))

print("\nKiitos tiedoista! Pieni hetki, lasken nyt optimipisteen ja myyntivoiton...")

# --------------------------
# Muuttujat (jatkuvat, ei-integerit linprogissa):
# x = [tuolit, pöydät], alarajat 0
# --------------------------

# Kohdefunktio (maksimointi):
# Max (tuoli_myyntihinta * tuolit + pöytä_myyntihinta * pöydät
#      - lego_pieni_hinta * (2 * tuolit + 2 * pöydät)
#      - lego_iso_hinta * (tuolit + 2 * pöydät))
#
# Kerroinmuodossa: Max (c_max · x), missä
# c_max = [
#   tuoli_myyntihinta - 2*lego_pieni_hinta - 1*lego_iso_hinta,
#   pöytä_myyntihinta - 2*lego_pieni_hinta - 2*lego_iso_hinta
# ]
c_max = np.array([
    tuoli_myyntihinta - 2 * lego_pieni_hinta - 1 * lego_iso_hinta,
    pöytä_myyntihinta - 2 * lego_pieni_hinta - 2 * lego_iso_hinta
], dtype=float)

# linprog minimoi, joten minimoidaan -c_max · x
c = -c_max

# --------------------------
# Rajoitteet
# --------------------------
# Pienet legot: 2*tuolit + 2*pöydät <= lego_pieni_määrä
# Isot legot:   1*tuolit + 2*pöydät <= lego_iso_määrä
A_ub = np.array([
    [2.0, 2.0],
    [1.0, 2.0],
])
b_ub = np.array([
    float(lego_pieni_määrä),
    float(lego_iso_määrä),
])

# Yhtäsuuruusrajoitteita ei ole
A_eq = None
b_eq = None

# Raja-arvot: tuolit >= 0, pöydät >= 0
bounds = [(0, None), (0, None)]

# --------------------------
# Ratkaistaan jatkuva LP (relaxaatio)
# --------------------------
res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,
              bounds=bounds, method='highs')

if not res.success:
    print("Optimointi epäonnistui:", res.message)
else:
    tuolit_opt, pöydät_opt = res.x
    # Palauta maksimiarvo: obj = -res.fun
    jatkuva_voitto = -res.fun

    # Tulostetaan optimiratkaisu (jatkuva)
    print("\nOptimiratkaisu (jatkuva, ei-integereitä):")
    print(f"Tuoleja: {tuolit_opt:.6g}")
    print(f"Pöytiä:  {pöydät_opt:.6g}")

    # Lasketaan myös alun perin määritelty myyntivoitto suoraan kaavalla varmistukseksi
    # (pitäisi vastata jatkuva_voitto)
    total_profit_cont = (
        tuoli_myyntihinta * tuolit_opt
        + pöytä_myyntihinta * pöydät_opt
        - lego_pieni_hinta * (2 * tuolit_opt + 2 * pöydät_opt)
        - lego_iso_hinta * (tuolit_opt + 2 * pöydät_opt)
    )
    print(f"\nMyyntivoitto (jatkuva): {total_profit_cont:.2f}€")

    # Jäikö jotain yli? (jatkuva)
    käytetyt_pienet = 2 * tuolit_opt + 2 * pöydät_opt
    käytetyt_isot = tuolit_opt + 2 * pöydät_opt

    lego_pieni_yli = lego_pieni_määrä - käytetyt_pienet
    lego_iso_yli = lego_iso_määrä - käytetyt_isot

    print(f"{lego_pieni_yli:.6g} kpl pieniä legoja jäi yli (jatkuva).")
    print(f"{lego_iso_yli:.6g} kpl isoja legoja jäi yli (jatkuva).")

    myyntihinta_cont = tuoli_myyntihinta * tuolit_opt + pöytä_myyntihinta * pöydät_opt
    print(f"\nMyyntihinta (jatkuva): {myyntihinta_cont:.2f}€")

# --------------------------
# Valinnainen: integer-optimi (brute force pienessä ongelmassa)
# Koska alkuperäinen malli oli kokonaislukuinen, etsitään paras kokonaisratkaisu
# käymällä läpi kaikki järkevät kombinaatiot (pieni hakutila).
# --------------------------
paras_voitto = -np.inf
paras_t = None
paras_p = None

# Turvallisesti voidaan iteroida 0..lego_pieni_määrä ja 0..lego_iso_määrä
for t in range(0, lego_pieni_määrä + 1):
    for p in range(0, lego_iso_määrä + 1):
        if (2*t + 2*p) <= lego_pieni_määrä and (t + 2*p) <= lego_iso_määrä:
            # Sama voittofunktio
            voitto = (
                tuoli_myyntihinta * t
                + pöytä_myyntihinta * p
                - lego_pieni_hinta * (2 * t + 2 * p)
                - lego_iso_hinta * (t + 2 * p)
            )
            if voitto > paras_voitto:
                paras_voitto = voitto
                paras_t = t
                paras_p = p

print("\nOptimiratkaisu (kokonaisluvut):")
print(f"Kannattaa valmistaa {paras_t} kpl tuoleja ja {paras_p} kpl pöytiä - tämä on optimitilanne!")
print(f"Jos tottelet ohjeitani, saavutat maksimaalisen myyntivoiton: {paras_voitto:.2f}€")

käytetyt_pienet_int = 2 * paras_t + 2 * paras_p
käytetyt_isot_int = paras_t + 2 * paras_p

print(f"{lego_pieni_määrä - käytetyt_pienet_int} kpl pieniä legoja jäi yli.")
print(f"{lego_iso_määrä - käytetyt_isot_int} kpl isoja legoja jäi yli.")